<a href="https://colab.research.google.com/github/Emie18/projetBDIAWEB/blob/main/ScriptKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [2]:
import pandas  as pd
accidents_file = "/content/gdrive/MyDrive/Projet_IA/export_IA2.csv"

In [3]:
def keep_selected_columns(data, columns_to_keep):
    # Suppression des colonnes qui ne sont pas dans la liste columns_to_keep
    columns_to_drop = set(data.columns) - set(columns_to_keep)
    data = data.drop(columns_to_drop, axis=1)
    return data

In [4]:
def convert_column_type(data, column, new_type):
    # Remplacement des virgules par des points dans la colonne
    data[column] = data[column].str.replace(',', '.')
    # Conversion du type de la colonne
    data[column] = data[column].astype(new_type)
    return data

In [20]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
import json

def knn_predict_accident(accident_info, accidents_file):
    # Charger le fichier CSV des accidents dans une DataFrame
    accidents_data = pd.read_csv(accidents_file, delimiter=";")

    accidents_data['descr_grav'] = accidents_data['descr_grav'].replace({
        'Indemne': 'pas grave',
        'Blessé léger': 'pas grave',
        'Blessé hospitalisé': 'grave',
        'Tué': 'grave'
    })
    columns_to_keep = ['descr_grav', 'latitude', 'longitude', 'descr_cat_veh',
                       'descr_agglo', 'descr_athmo', 'descr_lum', 'descr_etat_surf',
                       'descr_type_col']

    accidents_data = accidents_data[columns_to_keep]

    # Liste des colonnes à convertir
    columns_to_convert = ['descr_grav', 'descr_cat_veh', 'descr_agglo', 'descr_athmo', 'descr_lum',
                          'descr_etat_surf', 'descr_type_col']

    accidents_data = convert_column_type(accidents_data, "latitude",float)
    accidents_data = convert_column_type(accidents_data, "longitude",float)
    print("Accidents graves avant le label encoding :")
    accident_228 = accidents_data.iloc[228]
    print(accident_228)
    # Création de l'objet LabelEncoder
    le = LabelEncoder()
    for var in columns_to_convert:
        accidents_data[var] = le.fit_transform(accidents_data[var])
  
    print("Accident numéro 228 après le label encoding :")
    accident_228 = accidents_data.iloc[228]  # Accidents_data est indexé à partir de zéro, donc 227 correspond à l'indice 228
    print(accident_228)

    # Séparer les features (X) et les labels (y) à partir du jeu de données après le label encoding
    X = accidents_data.drop('descr_grav', axis=1)
    y = accidents_data['descr_grav']

    # Créer un modèle KNN avec k=5
    knn = KNeighborsClassifier(n_neighbors=5)

    # Ajuster le modèle aux données d'entraînement
    knn.fit(X, y)

    # Réorganiser les colonnes dans le jeu de données de test
    #accident_info_encoded = {}

    accident_info_reordered = pd.DataFrame.from_dict([accident_info], orient='columns')
    accident_info_reordered = accident_info_reordered[X.columns]

    # Prédire la classe de l'accident donné
    predicted_class = knn.predict(accident_info_reordered)
    print('predicted_classe')
    print(predicted_class)

    # Convertir la prédiction en "grave" ou "pas grave"
    predicted_class_string = "grave" if predicted_class[0] == 0 else "pas grave"

    # Retourner la classe de l'accident sous forme de JSON
    result = {'descr_grav': predicted_class_string}
    json_result = json.dumps(result)

    return json_result

# Exemple d'utilisation du script
accident_info = {
    'descr_cat_veh': 21,
    'descr_agglo': 1,
    'descr_lum': 4,
    'descr_athmo': 0,#0
    'descr_etat_surf': 8,#8
    'descr_type_col': 1,
    'latitude': 47.1167,
    'longitude': -2.1000
}  # Remplacez les features par les informations réelles de l'accident

predicted_class_json = knn_predict_accident(accident_info, accidents_file)
print(predicted_class_json)


Accidents graves avant le label encoding :
descr_grav                             grave
latitude                             47.1167
longitude                               -2.1
descr_cat_veh                        VL seul
descr_agglo               Hors agglomération
descr_athmo                          Normale
descr_lum                         Plein jour
descr_etat_surf                      Normale
descr_type_col     Deux véhicules - Frontale
Name: 228, dtype: object
Accident numéro 228 après le label encoding :
descr_grav          0.0000
latitude           47.1167
longitude          -2.1000
descr_cat_veh      21.0000
descr_agglo         1.0000
descr_athmo         3.0000
descr_lum           4.0000
descr_etat_surf     7.0000
descr_type_col      1.0000
Name: 228, dtype: float64
predicted_classe
[0]
{"descr_grav": "grave"}
